# 总体刚度矩阵

我们现在已经知道了一个单元的节点力和结点位移关系为
$$
F^{e_n}=K^{e_n} A^{e_n}\quad \mathrm{point} \in \mathrm{element}_n
$$
由于单元节点集是总体节点集的子集，所以我们有一选择函数，对于单元 $e_n$
$$
\begin{aligned}
&A=[u_0,v_0,u_1,v_1,\cdots,u_{N-1},v_{N-1}]^T\\
&\downarrow\mathrm{choose}[e_n]\\
&A^{e_n}=[u_i,v_i,u_j,v_j,u_k,v_k]^T
\end{aligned}
$$
我们可以用乘一个矩阵的方式写出该操作
$$
A^{e_n}=C^{e_n} A
$$
其中 $C^{e_n}$ 是一个 $6\times N$ 矩阵
$$
C^{e_n}=
\begin{matrix}
& 0 & \cdots & 1 & 0 & \cdots & 0 & 0 & \cdots & 0 & 0 & \cdots & 0 \\
& 0 & \cdots & 0 & 1 & \cdots & 0 & 0 & \cdots & 0 & 0 & \cdots & 0 \\
& 0 & \cdots & 0 & 0 & \cdots & 1 & 0 & \cdots & 0 & 0 & \cdots & 0 \\
& 0 & \cdots & 0 & 0 & \cdots & 0 & 1 & \cdots & 0 & 0 & \cdots & 0 \\
& 0 & \cdots & 0 & 0 & \cdots & 0 & 0 & \cdots & 1 & 0 & \cdots & 0 \\
& 0 & \cdots & 0 & 0 & \cdots & 0 & 0 & \cdots & 1 & 0 & \cdots & 0 \\
\hline
\mathrm{index:} & 0 & & 2i & 2i+1 & & 2j & 2j+1 & & 2k & 2k+1 & & N-1
\end{matrix}
$$

由于结点位移和节点力一一对应，所以我们可以求出总体的节点力有和结点位移类似的形式。注意这里的总体节点力是指该单元对总体节点力的贡献（一个节点的节点力由共享该节点的所有单元的贡献累加而成）
$$
F^{e_n}=C^{e_n} F \quad(\text{from }e_n)
$$
注意到
$$
(C^{e_n})^T C^{e_n}=
\left(
\begin{aligned}
1 &\quad \text{row of former equal to column of latter}\\
0 &\quad \text{other}
\end{aligned}
\right.
$$
所以
$$
(C^{e_n})^T C^{e_n}=\mathbf{I}_N
$$
所以
$$
(C^{e_n})^T F^{e_n}=(C^{e_n})^T C^{e_n} F
$$
即
$$
F=(C^{e_n})^T F^{e_n}
$$

由此我们可以列出总体节点力平衡方程。首先，单元的节点力为
$$
F^{e_n}=K^{e_n} A^{e_n}=K^{e_n} C^{e_n} A
$$
该单元对总体节点力的贡献为
$$
F=(C^{e_n})^T F^{e_n}=(C^{e_n})^T K^{e_n} C^{e_n} A
$$
考虑到各个单元的贡献，总体节点力矩阵为
$$
F=\sum_{n=0}^{N-1} (C^{e_n})^T K^{e_n} C^{e_n} A
$$
由于某一单元不影响总体位移矩阵的顺序，所以
$$
F=\left(\sum_{n=0}^{N-1} (C^{e_n})^T K^{e_n} C^{e_n}\right) A
$$
即
$$
F=K A
$$
可见整体刚度矩阵为
$$
K=\sum_{n=0}^{N-1} (C^{e_n})^T K^{e_n} C^{e_n}
$$

下面我们开始实现刚度矩阵的组装，为了让读者看得清楚。首先生成第0节“网格划分”中的网格。

In [5]:
import numpy as np
import pygmsh
with pygmsh.geo.Geometry() as geom:
    geom.add_polygon(
        [
            [0.0, 0.0],
            [1.0, 0.0],
            [1.0, 1.0],
            [0.0, 1.0],
        ],
        mesh_size=0.5,
    )
    mesh = geom.generate_mesh()
# 定义节点数据和网格数据
nodes=mesh.points
elements=mesh.cells_dict['triangle']

定义总体刚度矩阵的尺寸（2N方阵）

In [6]:
len_global=2*len(nodes)
K=np.zeros((len_global,len_global))

在迭代格式中，我们首先建立适当大小的零矩阵 $K^0$ ，然后在所有单元上迭代，每次更新
$$
K^n=K^{n-1}+(C^{e_n})^T K^{e_n} C^{e_n}
$$
这里 $(C^{e_n})^T (\bullet) C^{e_n}$ 相当于这样一种操作，将作用输入 $K^e$ 的局部指标位置的值，转换为输出 $K$ 的全局指标位置的值。这相当于将一个小矩阵分散为一个稀疏矩阵，所以我们当然不希望在内存中真的出现这样一个大矩阵，所以按照稀疏矩阵的思想，指出总体刚度矩阵元素的位置（值在局部刚度矩阵中）。

若某单元包括标号为 $i,j,k$ 的三个点，由于每个点有两个位移分量，所以该单元的局部指标和全局指标之间的联系如下：
$$
\begin{matrix}
i_{\mathrm{local}}  & 0 & 1 & 2 & 3 & 4 & 5 \\
i_{\mathrm{global}} & 2i & 2i+1 & 2j & 2j+1 & 2k & 2k+1
\end{matrix}
$$
所以总体刚度矩阵的值（即局部刚度矩阵）对应的总体刚度矩阵的位置是向量 $i_{\mathrm{global}}$ 和自身的外积
$$
\begin{bmatrix}
(2 i,2 i) & (2 i,2 i+1) & (2 i,2 j) & (2 i,2 j+1) & (2 i,2 k) & (2 i,2 k+1) \\
(2 i+1,2 i) & (2 i+1,2 i+1) & (2 i+1,2 j) & (2 i+1,2 j+1) & (2 i+1,2 k) & (2 i+1,2 k+1) \\
(2 j,2 i) & (2 j,2 i+1) & (2 j,2 j) & (2 j,2 j+1) & (2 j,2 k) & (2 j,2 k+1) \\
(2 j+1,2 i) & (2 j+1,2 i+1) & (2 j+1,2 j) & (2 j+1,2 j+1) & (2 j+1,2 k) & (2 j+1,2 k+1) \\
(2 k,2 i) & (2 k,2 i+1) & (2 k,2 j) & (2 k,2 j+1) & (2 k,2 k) & (2 k,2 k+1) \\
(2 k+1,2 i) & (2 k+1,2 i+1) & (2 k+1,2 j) & (2 k+1,2 j+1) & (2 k+1,2 k) & (2 k+1,2 k+1)
\end{bmatrix}
$$

为了理解方便，本节暂时不用真实的刚度矩阵，而用全1矩阵代替。

In [7]:
for element in elements:
    # 此处计算单元刚度矩阵，这里用全1矩阵代替
    K_element=np.ones((6,6))
    # 生成单元的全局指标索引
    deform_global_index=np.array([[2*element[i],2*element[i]+1] for i in range(3)],dtype=np.uint64).reshape(-1)
    # 生成索引的外积，并按索引将局部刚度矩阵的值填入总体刚度矩阵
    for i_local,i_global in enumerate(deform_global_index):
        for j_local,j_global in enumerate(deform_global_index):
            K[i_global,j_global]+=K_element[i_local,j_local]
print(f'K = \n{K}')

K = 
[[2. 2. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 2. 2. 0. 0. 0. 0.]
 [0. 0. 2. 2. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2.]
 [0. 0. 2. 2. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2.]
 [0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 2. 2. 0. 0.]
 [0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 2. 2. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 1. 1. 1. 1. 2. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 1. 1. 1. 1. 2. 2. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 3. 3. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 0. 0. 2. 2.]
 [1. 1. 1. 1. 0. 0. 0. 0. 3. 3. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 0. 0. 2. 2.]
 [0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 3. 3. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2.]
 [0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 3. 3. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 3. 3. 0. 0. 2. 2. 0. 0. 2. 2. 0. 0.]
 [0. 0. 0. 0. 1. 1. 